In [1]:
import os, fnmatch
import io
import csv
from read_data import convert_pdf_to_txt
from PIL import Image
import pytesseract
from wand.image import Image as wi
import hashlib
import pandas as pd
import numpy as np
import pprint as pp
from urllib.request import urlopen
import time
from datetime import datetime

In [2]:
def get_size(st):
    if os.path.isfile(st):
        return os.path.getsize(st)

In [3]:
def conv_pdf(pdf):
    if os.path.isfile(pdf):
        print('--> ' + str(datetime.now()) + ' ' + pdf)
        with open(pdf, 'rb') as f:
            # text from pdf
            bfr = io.BufferedReader(f)
            pdf_text = convert_pdf_to_txt(bfr)
            
            # recognized text (OCR) from pdf
            if len(pdf_text.strip()) <= 50:
                pdf_file = wi(filename=pdf, resolution=200)
                pdfImage = pdf_file.convert('jpeg')
                imageBlobs = []
                for img in pdfImage.sequence:
                    imgPage = wi(image=img)
                    imageBlobs.append(imgPage.make_blob('jpeg'))

                recognized_text = []

                for imgBlob in imageBlobs:
                    im = Image.open(io.BytesIO(imgBlob))
                    text = pytesseract.image_to_string(im, lang = 'eng')
                    recognized_text.append(text)

                recognized_text = '\n\n\n'.join(recognized_text)

            pdf_text = pdf_text if len(pdf_text.strip()) > 50 else recognized_text
            
            return pdf_text

In [4]:
def get_filename_by_url(url):
    file_id = url.split('=')[-1]
    
    def find(pattern):
        result = []
        for root, dirs, files in os.walk('./pdfs/'):
            for name in files:
                if fnmatch.fnmatch(name, pattern):
                    result.append(os.path.join(root, name))
        return result
    
    f_arr = find('*' + file_id + '.pdf')

    if len(f_arr) == 1:
        return f_arr[0]
    elif len(f_arr) > 1:
        raise ValueError('More than one file exist')
    elif len(f_arr) == 0:
        raise ValueError('No file')

In [5]:
maine_csv = pd.read_csv('maine_new.csv', index_col = 0)

In [6]:
maine_csv.describe(include='all')

,chamber,date,html,md5,session,state,text,topic,url,size,filename
count,13942,13942,0.0,13942,13942,13942,0.0,13942,13942,1.394200e+04,13942
unique,2,244,NaN,11970,1,1,NaN,1,13942,NaN,13942
top,House,Mon Mar 20 09:00:00 EDT 2017,NaN,047c371ef345de8d910f693e69b80f41,128th,maine,NaN,#TODO,https://legislature.maine.gov/legis/bills/getT...,NaN,"./pdfs/Williams, Lynne - 58882.pdf"
freq,9050,616,NaN,10,13942,13942,NaN,13942,1,NaN,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.193359e+05,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.512068e+05,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.178400e+04,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.007150e+04,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.176750e+04,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.182138e+05,NaN


In [7]:
maine_csv = maine_csv[12000:]

In [8]:
maine_csv.describe(include='all')

,chamber,date,html,md5,session,state,text,topic,url,size,filename
count,1942,1942,0.0,1942,1942,1942,0.0,1942,1942,1.942000e+03,1942
unique,2,207,NaN,1568,1,1,NaN,1,1942,NaN,1942
top,House,Wed Jan 17 13:31:00 EST 2018,NaN,5b37d6cb962fb5faaa043b555f44d58a,128th,maine,NaN,#TODO,https://legislature.maine.gov/legis/bills/getT...,NaN,"./pdfs/Melnick, Alysia - 80858.pdf"
freq,1223,90,NaN,10,1942,1942,NaN,1942,1,NaN,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.679625e+05,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.128248e+05,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.555050e+05,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.754808e+05,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.133990e+05,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.112322e+05,NaN


In [9]:
maine_csv['text'] = maine_csv.filename.apply(conv_pdf)

--> 2018-02-17 11:26:52.223394 ./pdfs/Thoyalil, Joby - 68287.pdf
--> 2018-02-17 11:26:52.974774 ./pdfs/Donovan, Anthony - 81505.pdf
--> 2018-02-17 11:26:53.844197 ./pdfs/Morneault, Patty - 67629.pdf
--> 2018-02-17 11:26:54.703813 ./pdfs/Schneider, Timothy - 41664.pdf
--> 2018-02-17 11:26:55.519187 ./pdfs/Millett, Rebecca - 44128.pdf
--> 2018-02-17 11:26:56.460150 ./pdfs/Hamann, Scott - 79136.pdf
--> 2018-02-17 11:26:57.347079 ./pdfs/Chipman, Benjamin - 43002.pdf
--> 2018-02-17 11:26:58.158423 ./pdfs/Austin, Jeffrey - 68658.pdf
--> 2018-02-17 11:26:59.325164 ./pdfs/Devin, Michael - 49357.pdf
--> 2018-02-17 11:27:00.140921 ./pdfs/Lovitch, Derek - 79272.pdf
--> 2018-02-17 11:27:00.738338 ./pdfs/Schaffer, Peggy - 71529.pdf
--> 2018-02-17 11:27:01.708819 ./pdfs/Ward, Polly - 57482.pdf
--> 2018-02-17 11:27:02.477661 ./pdfs/Stetkis, Joel - 73054.pdf
--> 2018-02-17 11:27:03.554055 ./pdfs/Carson, Brownie - 73257.pdf
--> 2018-02-17 11:27:04.478560 ./pdfs/Pouliot, Matthew - 46811.pdf
--> 2018-02-

--> 2018-02-17 11:30:01.201395 ./pdfs/Py, Jamie - 50556.pdf
--> 2018-02-17 11:30:02.450396 ./pdfs/Lund, William - 67335.pdf
--> 2018-02-17 11:30:03.313456 ./pdfs/Malaby, Richard - 77128.pdf
--> 2018-02-17 11:30:04.453372 ./pdfs/Healey, Dawn - 68965.pdf
--> 2018-02-17 11:30:05.756238 ./pdfs/Herbig, Erin - 81748.pdf
--> 2018-02-17 11:30:07.651447 ./pdfs/LeBlanc, Heidi - 67214.pdf
--> 2018-02-17 11:30:09.786931 ./pdfs/LeBlanc, Heidi - 67215.pdf
--> 2018-02-17 11:30:12.168938 ./pdfs/Jarrett, Richard - 75531.pdf
--> 2018-02-17 11:30:14.143797 ./pdfs/Montejo, Kathy - 79614.pdf
--> 2018-02-17 11:30:16.082104 ./pdfs/Gallant, Brenda - 82743.pdf
--> 2018-02-17 11:30:17.590335 ./pdfs/Hamm, Bethany - 78681.pdf
--> 2018-02-17 11:30:19.059216 ./pdfs/Hamm, Bethany - 78682.pdf
--> 2018-02-17 11:30:20.372747 ./pdfs/Hamm, Bethany - 78685.pdf
--> 2018-02-17 11:30:21.626219 ./pdfs/Hamm, Bethany - 78684.pdf
--> 2018-02-17 11:30:22.617258 ./pdfs/Hamm, Bethany - 78683.pdf
--> 2018-02-17 11:30:23.572089 ./pdf

--> 2018-02-17 11:34:33.271874 ./pdfs/Schneider, Hilary - 56370.pdf
--> 2018-02-17 11:34:34.660358 ./pdfs/Sylvester, Michael - 75522.pdf
--> 2018-02-17 11:34:36.244128 ./pdfs/Walker, Dan - 43593.pdf
--> 2018-02-17 11:34:37.561068 ./pdfs/Kilby-Chesley, Lois - 38894.pdf
--> 2018-02-17 11:34:38.901640 ./pdfs/Ackley, Kent - 67990.pdf
--> 2018-02-17 11:34:39.668857 ./pdfs/Ackley, Kent - 67991.pdf
--> 2018-02-17 11:34:40.446752 ./pdfs/Brautigam, Francis - 42122.pdf
--> 2018-02-17 11:34:41.680282 ./pdfs/Packard, Melissa - 69682.pdf
--> 2018-02-17 11:34:42.955537 ./pdfs/Doran, Dana - 81706.pdf
--> 2018-02-17 11:34:44.283888 ./pdfs/Vachon, Karen - 79119.pdf
--> 2018-02-17 11:34:45.468478 ./pdfs/Webber, Rebecca - 77253.pdf
--> 2018-02-17 11:34:46.591031 ./pdfs/Briar, Steve - 83924.pdf
--> 2018-02-17 11:35:06.925809 ./pdfs/Briar, Steve - 83922.pdf
--> 2018-02-17 11:35:28.316173 ./pdfs/Briar, Steve - 83927.pdf
--> 2018-02-17 11:35:49.147500 ./pdfs/Briar, Steve - 83929.pdf
--> 2018-02-17 11:36:08.9

--> 2018-02-17 11:43:01.361434 ./pdfs/Pelletier, Janice - 50335.pdf
--> 2018-02-17 11:43:02.930183 ./pdfs/Hallweaver, Taryn - 66840.pdf
--> 2018-02-17 11:43:04.328463 ./pdfs/Clemente, Frank - 71071.pdf
--> 2018-02-17 11:43:05.676049 ./pdfs/Joyce, Mark - 77843.pdf
--> 2018-02-17 11:43:06.934085 ./pdfs/Spear, John - 38758.pdf
--> 2018-02-17 11:43:07.919500 ./pdfs/Flynn, Julie - 53871.pdf
--> 2018-02-17 11:43:09.313765 ./pdfs/Collins, Paulina - 46702.pdf
--> 2018-02-17 11:43:10.453921 ./pdfs/Smith, George - 43811.pdf
--> 2018-02-17 11:43:30.060946 ./pdfs/Wynne, Elissa - 71384.pdf
--> 2018-02-17 11:43:31.239128 ./pdfs/Williams, Scott - 80794.pdf
--> 2018-02-17 11:43:32.639832 ./pdfs/Poliquin, Greg - 78695.pdf
--> 2018-02-17 11:43:34.042272 ./pdfs/Eaton, Julie - 68869.pdf
--> 2018-02-17 11:43:35.107503 ./pdfs/Cioppa, Eric - 75862.pdf
--> 2018-02-17 11:43:36.402247 ./pdfs/Eaton, Julie - 79297.pdf
--> 2018-02-17 11:43:37.730866 ./pdfs/Langley, Brian - 69692.pdf
--> 2018-02-17 11:43:40.862584 

--> 2018-02-17 11:48:34.809615 ./pdfs/Collins, Paulina - 76661.pdf
--> 2018-02-17 11:48:36.096680 ./pdfs/Foley, Robert - 69532.pdf
--> 2018-02-17 11:48:37.636818 ./pdfs/Cox, Tamara - 84004.pdf
--> 2018-02-17 11:48:39.075749 ./pdfs/Novak, Sarah - 71356.pdf
--> 2018-02-17 11:48:40.336383 ./pdfs/Novak, Sarah - 71354.pdf
--> 2018-02-17 11:48:41.388993 ./pdfs/Novak, Sarah - 71355.pdf
--> 2018-02-17 11:48:42.487678 ./pdfs/Soltan, Charlie - 78360.pdf
--> 2018-02-17 11:48:44.877325 ./pdfs/Maurer, Jessica - 77521.pdf
--> 2018-02-17 11:48:46.029789 ./pdfs/Smith, Gordon - 46173.pdf
--> 2018-02-17 11:48:47.555890 ./pdfs/Smith, George - 69865.pdf
--> 2018-02-17 11:48:49.471558 ./pdfs/Riley, Dan - 57004.pdf
--> 2018-02-17 11:48:50.821332 ./pdfs/Little, David - 61540.pdf
--> 2018-02-17 11:48:52.052142 ./pdfs/Stange, Kevin - 80125.pdf
--> 2018-02-17 11:49:38.439391 ./pdfs/Doak, Gwen - 80736.pdf
--> 2018-02-17 11:49:40.180871 ./pdfs/Brautigam, Francis - 43818.pdf
--> 2018-02-17 11:49:41.588132 ./pdfs/G

...

--> 2018-02-17 12:36:57.165432 ./pdfs/Tipping, Ryan - 47836.pdf
--> 2018-02-17 12:37:34.064155 ./pdfs/Van Tuinen, William - 80686.pdf
--> 2018-02-17 12:37:36.899829 ./pdfs/Shephard, Samantha - 78518.pdf
--> 2018-02-17 12:38:40.855333 ./pdfs/Hohenwarter, John - 71433.pdf
--> 2018-02-17 12:38:44.141083 ./pdfs/Amarasingham, Oamshri - 42612.pdf
--> 2018-02-17 12:38:47.077402 ./pdfs/Coleman, Diane - 67842.pdf
--> 2018-02-17 12:38:48.837562 ./pdfs/Coleman, Diane - 67841.pdf
--> 2018-02-17 12:38:50.110706 ./pdfs/Paquette, Lydia - 78599.pdf
--> 2018-02-17 12:38:54.720099 ./pdfs/Luchini, Louis - 38534.pdf
--> 2018-02-17 12:38:57.135783 ./pdfs/Cohen, Jim - 84644.pdf
--> 2018-02-17 12:39:06.335952 ./pdfs/Santarelli, Vanessa - 78639.pdf
--> 2018-02-17 12:39:08.864042 ./pdfs/Schneider, Hilary - 78557.pdf
--> 2018-02-17 12:39:11.158983 ./pdfs/Shaw, Helen - 70145.pdf
--> 2018-02-17 12:39:14.274387 ./pdfs/Woloson, Ann - 79012.pdf
--> 2018-02-17 12:39:16.971856 ./pdfs/Woloson, Ann - 79013.pdf
--> 2018-

--> 2018-02-17 12:55:06.944990 ./pdfs/Voorhees, Dylan - 72700.pdf
--> 2018-02-17 12:55:10.108785 ./pdfs/Amarasingham, Oamshri - 79894.pdf
--> 2018-02-17 12:55:12.884625 ./pdfs/Yudenfreund, Tamar - 67355.pdf
--> 2018-02-17 12:55:15.701270 ./pdfs/Yudenfreund, Tamar - 67354.pdf
--> 2018-02-17 12:55:18.567272 ./pdfs/Volk, Amy - 84244.pdf
--> 2018-02-17 12:55:21.539784 ./pdfs/Gervais, Joe - 66738.pdf
--> 2018-02-17 12:55:25.201820 ./pdfs/Giles, Jon - 80902.pdf
--> 2018-02-17 12:55:28.109046 ./pdfs/Kinney, Jonathan - 38672.pdf
--> 2018-02-17 12:55:31.143149 ./pdfs/Clifford, James - 77242.pdf
--> 2018-02-17 12:55:35.623723 ./pdfs/Rogers, Gemma - 84206.pdf
--> 2018-02-17 12:55:38.802336 ./pdfs/Rogers, Gemma - 84203.pdf
--> 2018-02-17 12:55:41.858538 ./pdfs/Rogers, Gemma - 84205.pdf
--> 2018-02-17 12:55:44.867072 ./pdfs/Rogers, Gemma - 84200.pdf
--> 2018-02-17 12:55:47.732212 ./pdfs/Rogers, Gemma - 84201.pdf
--> 2018-02-17 12:55:50.581006 ./pdfs/Rogers, Gemma - 84202.pdf
--> 2018-02-17 12:55:53

--> 2018-02-17 13:19:23.101652 ./pdfs/Wilson, Jennifer - 50621.pdf
--> 2018-02-17 13:19:26.119932 ./pdfs/Martin, Garrett - 68108.pdf
--> 2018-02-17 13:19:29.018917 ./pdfs/Martin, Garrett - 68107.pdf
--> 2018-02-17 13:19:31.842699 ./pdfs/Martin, Garrett - 68106.pdf
--> 2018-02-17 13:19:34.589365 ./pdfs/Martin, Garrett - 68105.pdf
--> 2018-02-17 13:19:37.331583 ./pdfs/Martin, Garrett - 68110.pdf
--> 2018-02-17 13:19:40.162161 ./pdfs/Martin, Garrett - 68109.pdf
--> 2018-02-17 13:19:43.062499 ./pdfs/Martin, Garrett - 68111.pdf
--> 2018-02-17 13:19:45.908560 ./pdfs/Hanna, Jim - 79046.pdf
--> 2018-02-17 13:19:47.868721 ./pdfs/Hanna, Jim - 79047.pdf
--> 2018-02-17 13:19:49.803706 ./pdfs/Hanna, Jim - 79048.pdf
--> 2018-02-17 13:19:51.748387 ./pdfs/Hambleton, Paul - 77105.pdf
--> 2018-02-17 13:19:53.853109 ./pdfs/Hambleton, Paul - 77107.pdf
--> 2018-02-17 13:19:56.095411 ./pdfs/Hambleton, Paul - 77106.pdf
--> 2018-02-17 13:19:58.359572 ./pdfs/Hambleton, Paul - 77108.pdf
--> 2018-02-17 13:20:00.

--> 2018-02-17 13:34:43.458723 ./pdfs/Mehnert, Jenna - 77815.pdf
--> 2018-02-17 13:34:46.571562 ./pdfs/O'Donal, Jeffrey - 72525.pdf
--> 2018-02-17 13:34:48.365486 ./pdfs/Sebell, Vicki - 50306.pdf
--> 2018-02-17 13:34:55.393422 ./pdfs/Mansins, Donald - 78381.pdf
--> 2018-02-17 13:36:20.318907 ./pdfs/Cronin, Nancy - 78600.pdf
--> 2018-02-17 13:36:24.799970 ./pdfs/McInnis, Marcia - 82964.pdf
--> 2018-02-17 13:36:34.395429 ./pdfs/Dorie, Laura - 39497.pdf
--> 2018-02-17 13:36:37.070561 ./pdfs/Didisheim, Peter - 80822.pdf
--> 2018-02-17 13:36:40.741519 ./pdfs/LaBrecque, James - 79748.pdf
--> 2018-02-17 13:36:43.428690 ./pdfs/Rabinowitz, Julie - 46890.pdf
--> 2018-02-17 13:36:46.457184 ./pdfs/Coffin, Dawn - 79174.pdf
--> 2018-02-17 13:38:08.909024 ./pdfs/Braden, Twain - 82116.pdf
--> 2018-02-17 13:38:12.750212 ./pdfs/Woodruff, Scott - 75333.pdf
--> 2018-02-17 13:38:16.252120 ./pdfs/LaCasse, Jeff - 39697.pdf
--> 2018-02-17 13:40:37.051844 ./pdfs/Cloutier, Chris - 43035.pdf
--> 2018-02-17 13:40

--> 2018-02-17 17:22:47.108259 ./pdfs/Adolphsen, Nicholas - 78585.pdf
--> 2018-02-17 17:23:55.135082 ./pdfs/Adolphsen, Nicholas - 78586.pdf
--> 2018-02-17 17:24:47.348204 ./pdfs/Gauvin, Paul - 67573.pdf
--> 2018-02-17 17:25:36.899980 ./pdfs/Hewitt, Bill - 84966.pdf
--> 2018-02-17 17:33:30.650277 ./pdfs/Savage, Rick - 84250.pdf
--> 2018-02-17 17:38:17.227367 ./pdfs/Sampson, Heidi - 84350.pdf
--> 2018-02-17 17:38:34.296160 ./pdfs/Bear, Henry John - 77281.pdf
--> 2018-02-17 17:38:56.860826 ./pdfs/Brown, Ashley - 79761.pdf
--> 2018-02-17 17:39:23.953723 ./pdfs/Pettengill, Chantel - 84219.pdf
--> 2018-02-17 17:41:26.674574 ./pdfs/Pettengill, Chantel - 84222.pdf
--> 2018-02-17 17:43:31.814386 ./pdfs/Pettengill, Chantel - 84225.pdf
--> 2018-02-17 17:45:36.613453 ./pdfs/Pettengill, Chantel - 84220.pdf
--> 2018-02-17 17:47:40.979420 ./pdfs/Pettengill, Chantel - 84221.pdf
--> 2018-02-17 17:49:49.252918 ./pdfs/Pettengill, Chantel - 84224.pdf
--> 2018-02-17 17:51:56.407510 ./pdfs/Pettengill, Chant

In [10]:
maine_csv

,chamber,date,html,md5,session,state,text,topic,url,size,filename
bill_name,,,,,,,,,,,
"LD 971, HP 684",House,Wed Apr 5 13:00:00 EDT 2017,NaN,bb6a1ca54b5ff31d4b9c538b5de14c3f,128th,maine,   \n  \n \n \n\n' \n\n126 Sewall Street \nAug...,#TODO,https://legislature.maine.gov/legis/bills/getT...,155505,"./pdfs/Thoyalil, Joby - 68287.pdf"
"LD 1686, SP 622",House & Senate,Thu Jan 11 13:00:00 EST 2018,NaN,8d3fb90f9f00f0adff43bf2fcf948e9e,128th,maine,                \n\nw SIERRA \nCLUB \n\nMame C...,#TODO,https://legislature.maine.gov/legis/bills/getT...,155536,"./pdfs/Donovan, Anthony - 81505.pdf"
"LD 1149, HP 812",House,Tue Apr 11 13:30:00 EDT 2017,NaN,9fdab7e2e56cc787c4fd0b09cfe3051d,128th,maine,      \n                     \n  \n \n  \n \n ...,#TODO,https://legislature.maine.gov/legis/bills/getT...,155543,"./pdfs/Morneault, Patty - 67629.pdf"
"LD 529, HP 373",House,Wed Mar 1 09:00:00 EST 2017,NaN,dc1855dc07dd66670d77ad0e687a3e1a,128th,maine,"        \n\nLD 529 \n\nl \n\nA""A°ttlZt'§.Zsé'|...",#TODO,https://legislature.maine.gov/legis/bills/getT...,155604,"./pdfs/Schneider, Timothy - 41664.pdf"
"LD 38, SP 17",House & Senate,Tue Feb 14 13:00:00 EST 2017,NaN,ef9e386beefec6d3ad13458aaf8f1a34,128th,maine,                                    \n\n/251% ...,#TODO,https://legislature.maine.gov/legis/bills/getT...,155630,"./pdfs/Millett, Rebecca - 44128.pdf"
"LD 1063, HP 746",House,Fri Apr 28 09:00:00 EDT 2017,NaN,96d7e3c163ea839d64aa74fd9f9a8bea,128th,maine,HoUsE or REPRESENTATIVES \n\n2 STATE HOUSE STA...,#TODO,https://legislature.maine.gov/legis/bills/getT...,155723,"./pdfs/Hamann, Scott - 79136.pdf"
"LD 450, SP 152",House & Senate,Wed Mar 8 10:00:00 EST 2017,NaN,76fbc1f0446f8566ea068cfe6f116d4f,128th,maine,$7 7 \n\n__ \n\ng \n\n1 \n\nS \n\no \n\n- \n\n...,#TODO,https://legislature.maine.gov/legis/bills/getT...,155724,"./pdfs/Chipman, Benjamin - 43002.pdf"
"LD 445, SP 147",House & Senate,Thu Apr 13 13:00:00 EDT 2017,NaN,666be5f62174d63f0392e9f576fc58f5,128th,maine, \n \n\nA TESTIMONY OF THE MAINE HOSPITAL ASS...,#TODO,https://legislature.maine.gov/legis/bills/getT...,155741,"./pdfs/Austin, Jeffrey - 68658.pdf"
"LD 703, HP 494",House,Wed Mar 22 10:00:00 EDT 2017,NaN,fad9e40602926a58dbf403ec45ee6bea,128th,maine,HoUsE or REPRESENTATIVES \n\n2 STATE HOUSE STA...,#TODO,https://legislature.maine.gov/legis/bills/getT...,155744,"./pdfs/Devin, Michael - 49357.pdf"


In [11]:
maine_csv.loc[:, ['chamber','date','html','md5','session','state','text','topic','filename']].to_csv('maine-pt9.csv')

In [ ]:
maine_csv['size'] = maine_csv.url.apply(get_filename_by_url).apply(get_size)
maine_csv['filename'] = maine_csv.url.apply(get_filename_by_url)
maine_csv = maine_csv.sort_values('size')